# ReverseNumberFlow example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread

In [3]:
FLOW_MODULES_PATH = "./"

# Connect to CoLink server

To connect to a real server, run below cell instead of the one after.

In [4]:
# USER_JWT_PATH = "/home/staverm/workspace/coflows-dev/coflows/jwts.txt"
# user_jwt = open(USER_JWT_PATH).readline().strip()
# cl = CoLink("http://127.0.0.1:2021", user_jwt)

In [5]:
cl = colink_utils.start_colink_server()

# Load flow config

Every flow instance gets initialized with a flow config specified in a yaml file.

The ``_target_`` key specifies the method name used to instantiate the flow. 

## Load config for ReverseNumberAtomicFlow

In [6]:
reverse_number_atomic_default_config = read_yaml_file(
    os.path.join(FLOW_MODULES_PATH, "ReverseNumberFlowModule/ReverseNumberAtomicFlow.yaml"))
print(json.dumps(reverse_number_atomic_default_config, indent=4))

{
    "name": "ReverseNumber",
    "description": "A flow that takes in a number and reverses it.",
    "_target_": "ReverseNumberFlowModule.ReverseNumberAtomicFlow.instantiate_from_default_config",
    "input_interface": {
        "_target_": "aiflows.interfaces.KeyInterface",
        "keys_to_select": [
            "number"
        ]
    },
    "output_interface": {
        "_target_": "aiflows.interfaces.KeyInterface",
        "keys_to_rename": {
            "output_number": "reversed_number"
        }
    }
}


## Load config for ReverseNumberSequentialFlow

In [7]:
reverse_number_sequential_default_config = read_yaml_file(
    os.path.join(FLOW_MODULES_PATH, "ReverseNumberFlowModule/ReverseNumberSequentialFlow.yaml"))
print(json.dumps(reverse_number_sequential_default_config, indent=4))

{
    "name": "ReverseNumberTwice",
    "description": "A sequential flow that reverses a number twice.",
    "_target_": "ReverseNumberFlowModule.ReverseNumberSequentialFlow.instantiate_from_default_config",
    "flow_type": "ReverseNumberSequentialFlow_served",
    "input_interface": [
        "number"
    ],
    "output_interface": [
        "output_number"
    ],
    "subflows_config": {
        "first_reverse_flow": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_type": "ReverseNumberAtomicFlow_served",
            "name": "ReverseNumberFirst",
            "description": "A flow that takes in a number and reverses it."
        },
        "second_reverse_flow": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_type": "ReverseNumberAtomicFlow_served",
            "name": "ReverseNumberSecond",
        

# Serve Flows

In [8]:
serve_utils.serve_flow(
    cl=cl,
    flow_type="ReverseNumberAtomicFlow_served",
    default_config=reverse_number_atomic_default_config,
    default_state=None,
    default_dispatch_point="coflows_dispatch",
    serve_mode="statefull",
)
serve_utils.serve_flow(
    cl=cl,
    flow_type="ReverseNumberSequentialFlow_served",
    default_config=reverse_number_sequential_default_config,
    default_state=None,
    default_dispatch_point="coflows_dispatch",
    serve_mode="statefull",
)

Started serving at flows:ReverseNumberAtomicFlow_served.
Started serving at flows:ReverseNumberSequentialFlow_served.


True

# Start dispatch workers

In [9]:
run_dispatch_worker_thread(cl, dispatch_point="coflows_dispatch", flow_modules_base_path=FLOW_MODULES_PATH)

Dispatch worker started in attached thread.

~~~ Dispatch task ~~~
flow_type: ReverseNumberSequentialFlow_served
flow_id: 882b3caf-e425-4218-98ad-239542f83acf
message_ids: ['de491e76-b3ae-468c-9314-cea0a04598c5']

Called ReverseNumberSequential



In [10]:
run_dispatch_worker_thread(cl, dispatch_point="coflows_dispatch", flow_modules_base_path=FLOW_MODULES_PATH)

Dispatch worker started in attached thread.

~~~ Dispatch task ~~~
flow_type: ReverseNumberAtomicFlow_served
flow_id: dbb911c8-e3f2-4441-9f43-5b899905b150
message_ids: ['876ba971-520e-4347-b096-0dfeecb6c944']

Called ReverseNumberAtomic


~~~ Dispatch task ~~~
flow_type: ReverseNumberAtomicFlow_served
flow_id: f9af6ba5-b5b8-4b78-a93c-f148b665f2bd
message_ids: ['d26e9f48-d549-454f-9841-a5422bbf8502']

Called ReverseNumberAtomic



# Get flow instance

In [11]:
flow = serve_utils.recursive_mount(
    cl=cl,
    client_id="local",
    flow_type="ReverseNumberSequentialFlow_served",
    config_overrides=None,
    initial_state=None,
    dispatch_point_override=None,
)

Mounted dbb911c8-e3f2-4441-9f43-5b899905b150 at flows:ReverseNumberAtomicFlow_served:mounts:local:dbb911c8-e3f2-4441-9f43-5b899905b150
Mounted f9af6ba5-b5b8-4b78-a93c-f148b665f2bd at flows:ReverseNumberAtomicFlow_served:mounts:local:f9af6ba5-b5b8-4b78-a93c-f148b665f2bd
Mounted 882b3caf-e425-4218-98ad-239542f83acf at flows:ReverseNumberSequentialFlow_served:mounts:local:882b3caf-e425-4218-98ad-239542f83acf


In [12]:
print("Pushing to...\n", flow)
  
input_data = {"id": 0, "number": 81}
    
input_message = FlowMessage(
    data=input_data,
    src_flow="Coflows team",
    dst_flow=flow,
    is_input_msg=True,
)

flow.ask(input_message).get_data()

Pushing to...
 Name: Proxy_ReverseNumberSequentialFlow_served
Class name: AtomicFlow
Type: AtomicFlow
Description: Proxy of ReverseNumberSequentialFlow_served
Input keys: no input keys
Output keys: no output keys



{'output_number': 81}

# Unserve flow, delete instances

In [13]:
def status():
    colink_utils.print_served_flows(cl)
    print("\nLive flow instances:")
    colink_utils.print_flow_instances(cl)
status()

 ReverseNumberAtomicFlow_served/
   init
   serve_mode
   default_dispatch_point
   default_config
   mounts/
     local/
       dbb911c8-e3f2-4441-9f43-5b899905b150/
         state
         config_overrides
         init
       f9af6ba5-b5b8-4b78-a93c-f148b665f2bd/
         init
         config_overrides
         state
 ReverseNumberSequentialFlow_served/
   serve_mode
   default_config
   init
   default_dispatch_point
   mounts/
     local/
       882b3caf-e425-4218-98ad-239542f83acf/
         config_overrides
         state
         init

Live flow instances:
 dbb911c8-e3f2-4441-9f43-5b899905b150
 f9af6ba5-b5b8-4b78-a93c-f148b665f2bd
 882b3caf-e425-4218-98ad-239542f83acf


In [14]:
serve_utils.unserve_flow(cl=cl, flow_type="ReverseNumberAtomicFlow_served")

ReverseNumberAtomicFlow_served has been unserved.


In [15]:
serve_utils.is_flow_served(cl, "ReverseNumberAtomicFlow_served")

False

In [16]:
status()

 ReverseNumberAtomicFlow_served/
   init
   serve_mode
   default_dispatch_point
   default_config
   mounts/
     local/
       dbb911c8-e3f2-4441-9f43-5b899905b150/
         state
         config_overrides
         init
       f9af6ba5-b5b8-4b78-a93c-f148b665f2bd/
         init
         config_overrides
         state
 ReverseNumberSequentialFlow_served/
   serve_mode
   default_config
   init
   default_dispatch_point
   mounts/
     local/
       882b3caf-e425-4218-98ad-239542f83acf/
         config_overrides
         state
         init

Live flow instances:
 dbb911c8-e3f2-4441-9f43-5b899905b150
 f9af6ba5-b5b8-4b78-a93c-f148b665f2bd
 882b3caf-e425-4218-98ad-239542f83acf


In [17]:
serve_utils.delete_served_flow(cl, "ReverseNumberAtomicFlow_served")

Deleted flow instance dbb911c8-e3f2-4441-9f43-5b899905b150
Deleted flow instance f9af6ba5-b5b8-4b78-a93c-f148b665f2bd
Stopped serving at flows:ReverseNumberAtomicFlow_served


In [18]:
status()

 ReverseNumberAtomicFlow_served/
   mounts/
     local/
       dbb911c8-e3f2-4441-9f43-5b899905b150/
       f9af6ba5-b5b8-4b78-a93c-f148b665f2bd/
 ReverseNumberSequentialFlow_served/
   serve_mode
   default_config
   init
   default_dispatch_point
   mounts/
     local/
       882b3caf-e425-4218-98ad-239542f83acf/
         config_overrides
         state
         init

Live flow instances:
 882b3caf-e425-4218-98ad-239542f83acf


In [21]:
serve_utils.delete_flow_instance(cl, flow.get_instance_id())

Deleted flow instance 882b3caf-e425-4218-98ad-239542f83acf.


In [22]:
status()

 ReverseNumberAtomicFlow_served/
   mounts/
     local/
       dbb911c8-e3f2-4441-9f43-5b899905b150/
       f9af6ba5-b5b8-4b78-a93c-f148b665f2bd/
 ReverseNumberSequentialFlow_served/
   serve_mode
   default_config
   init
   default_dispatch_point
   mounts/
     local/
       882b3caf-e425-4218-98ad-239542f83acf/

Live flow instances:
